## Package Imports

In [1]:
import base64
import json
import os
import platform
import requests
import tarfile
import zipfile
import uuid
import pandas as pd

from IPython.core.display import display, HTML

## Setup CPD CTL

In [2]:
#DOWNLOAD CPDCTL

PLATFORM = platform.system().lower()
CPDCTL_ARCH = "{}_amd64".format(PLATFORM)
CPDCTL_RELEASES_URL="https://api.github.com/repos/IBM/cpdctl/releases"
CWD = os.getcwd()
PATH = os.environ['PATH']
CPDCONFIG = os.path.join(CWD, '.cpdctl.config.yml')

response = requests.get(CPDCTL_RELEASES_URL)
assets = response.json()[0]['assets']
platform_asset = next(a for a in assets if CPDCTL_ARCH in a['name'])
cpdctl_url = platform_asset['url']
cpdctl_file_name = platform_asset['name']

response = requests.get(cpdctl_url, headers={'Accept': 'application/octet-stream'})
with open(cpdctl_file_name, 'wb') as f:
    f.write(response.content)
    
display(HTML('<code>cpdctl</code> binary downloaded from: <a href="{}">{}</a>'.format(platform_asset['browser_download_url'], platform_asset['name'])))

In [3]:
%%capture

%env PATH={CWD}:{PATH}
%env CPDCONFIG={CPDCONFIG}

In [4]:
if cpdctl_file_name.endswith('tar.gz'):
    with tarfile.open(cpdctl_file_name, "r:gz") as tar:
        tar.extractall()
elif cpdctl_file_name.endswith('zip'):
    with zipfile.ZipFile(cpdctl_file_name, 'r') as zf:
        zf.extractall()

if CPDCONFIG and os.path.exists(CPDCONFIG):
    os.remove(CPDCONFIG)
    
version_r = ! cpdctl version
CPDCTL_VERSION = version_r.s

print("cpdctl version: {}".format(CPDCTL_VERSION))

cpdctl version: 1.0.79


In [5]:
CPD_URL = 'https://zen-cpd-zen.cpd-msk-dal10-2bef1f4b4097001da9502000c44fc2b2-0000.us-south.containers.appdomain.cloud'

In [6]:
! cpdctl config profile set cpd --url {CPD_URL}

In [8]:
#! cpdctl config user set cpd_admin --username {CPD_USER_NAME} --password {CPD_USER_PASSWORD}
#! cpdctl config context set cpd --profile cpd --user cpd_user
#! cpdctl config context list
#! cpdctl config context use cpd
#! cpdctl project list --output json

## Get Project Source

In [7]:
PROJECT_ZIP_FILE_NAME = 'BreastCancerClassification.zip'

In [8]:
!rm BreastCancerClassification.zip
!wget https://github.com/ricdodds/ml101/raw/main/projects/BreastCancerClassification.zip
! unzip -l BreastCancerClassification.zip

rm: cannot remove 'BreastCancerClassification.zip': No such file or directory
--2021-06-17 17:31:19--  https://github.com/ricdodds/ml101/raw/main/projects/BreastCancerClassification.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ricdodds/ml101/main/projects/BreastCancerClassification.zip [following]
--2021-06-17 17:31:19--  https://raw.githubusercontent.com/ricdodds/ml101/main/projects/BreastCancerClassification.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84385 (82K) [application/zip]
Saving to: ‘BreastCancerClassification.zip’

BreastCancerClassif 100%[===================>]  82.41K

## Create Project for Users

In [9]:
PROJECT_NAME = 'BreastCancerClassification'

In [10]:
# GET PROJECTS AND DELETE ANY WITH SAME IMPORT NAME. MAY SWITCH TO CREATE PROJECT WITH NAME + SEQUENCE NUMBER
def delete_project(project_name):
    JMES_QUERY = "resources[?entity.name == '{}'].metadata.guid".format(project_name)
    result = ! cpdctl project list --output json --jmes-query "{JMES_QUERY}"
    project_ids = json.loads(result.s)
    if project_ids:
        for project_id in project_ids:
            print('Deleting project with ID: {}'.format(project_id))
            ! cpdctl project delete --project-id {project_id}

In [11]:
def create_project(project_name):
    STORAGE = {"type": "assetfiles", "guid": str(uuid.uuid4())}
    STORAGE_JSON = json.dumps(STORAGE)
    result = ! cpdctl project create --name {project_name} --output json --raw-output --storage '{STORAGE_JSON}' --jmes-query 'location'
    res_project_id = result.s.split('/')[-1]
    print("Created new '{}' project. ID is: {}".format(project_name, res_project_id))
    return res_project_id

In [12]:
def import_zip_assets(project_id, project_file_name):
    result = ! cpdctl asset import start --project-id {project_id} --import-file {project_file_name} --output json --jmes-query "metadata.id" --raw-output
    import_id = result.s
    print("Import project zip. ID is: {}".format(import_id))
    return import_id

In [13]:
def check_asset_import_state(project_id, import_id):
    ! cpdctl asset import get --project-id {project_id} --import-id {import_id}


In [14]:
def list_project_assets(project_id):
    ! cpdctl asset search --query '*:*' --type-name asset --project-id {project_id}


In [15]:
def validate_project_created(project_name):
    res = ! cpdctl project list --output json
    projects = json.loads(res.s)
    project_found = False
    for project in projects['resources']:
        if project_name == project['entity']['name']:
            project_found = True
            break
    return project_found

In [16]:
df = pd.read_csv('/project_data/data_asset/passwords.csv')
#d = {'username': ['demouser1', 'demouser2'], 'password': ['demouser1', 'demouser2'], 'email': ['jrtorres@us.ibm.com', 'jrtorres@us.ibm.com']}
#df = pd.DataFrame(data=d)
df.head(10)

username           password              email
0  user000    81-none-hole-41  user000@email.com
1  user001  26-count-level-05  user001@email.com
2  user002    07-loss-arms-35  user002@email.com
3  user003   91-brown-king-51  user003@email.com
4  user004  06-shine-clear-60  user004@email.com
5  user005  03-large-horse-92  user005@email.com
6  user006   26-wore-track-17  user006@email.com
7  user007    77-jump-half-46  user007@email.com
8  user008    76-left-shoe-67  user008@email.com
9  user009    99-grew-says-67  user009@email.com

In [ ]:
for index, row in df.iterrows():
    user = 'cpd_user_' + str(index)
    context = 'cpd_cont_user_' + str(index)
    ! cpdctl config user set {user} --username {row['username']} --password {row['password']}
    ! cpdctl config context set {context} --profile cpd --user {user}
    ! cpdctl config context use {context}
    delete_project(PROJECT_NAME)
    proj_id = create_project(PROJECT_NAME)
    imp_id = import_zip_assets(proj_id, PROJECT_ZIP_FILE_NAME)
    check_asset_import_state(proj_id, imp_id)
    proj_created = validate_project_created(PROJECT_NAME)
    list_project_assets(proj_id)

Switched to context "cpd_cont_user_0".
Created new 'BreastCancerClassification' project. ID is: d54c21eb-d777-4755-afd2-7cc0aff8f007
Import project zip. ID is: 506f36f9-7f3c-4254-8ef0-039c4664b93c
...
              
ID:        506f36f9-7f3c-4254-8ef0-039c4664b93c   
Created:   2021-06-17T17:31:35.760Z   
State:     completed   
...
ID                                     Name                                 Created                    Description   Type         State       Tags         Size   
66f2f4b1-b62d-43c1-aa77-1e43a48c3711   data.csv                             2021-06-17T17:31:40.000Z                 data_asset   available   []           125204   
99b9963b-ca61-4789-a8ea-bea51a54c4a3   breast_cancer_tumor_classification   2021-06-17T17:31:40.000Z                 notebook     available   [notebook]   7911   
Switched to context "cpd_cont_user_1".
Created new 'BreastCancerClassification' project. ID is: 408e0f60-f570-4ace-830b-0dd97d43a7c1
Import project zip. ID is: 9d27344d-3504-4

Switched to context "cpd_cont_user_9".
Created new 'BreastCancerClassification' project. ID is: 6909347b-a7a0-4ea8-a350-772db84e7177
Import project zip. ID is: 4dbb5914-7812-413d-b98d-8d5ed38860ba
...
              
ID:        4dbb5914-7812-413d-b98d-8d5ed38860ba   
Created:   2021-06-17T17:34:07.352Z   
State:     completed   
...
ID                                     Name                                 Created                    Description   Type         State       Tags         Size   
74348817-40a0-4109-a474-d78603a961d0   data.csv                             2021-06-17T17:34:12.000Z                 data_asset   available   []           125204   
0d358756-cf8a-45ad-9a01-1b3253eff24b   breast_cancer_tumor_classification   2021-06-17T17:34:13.000Z                 notebook     available   [notebook]   7911   
Switched to context "cpd_cont_user_10".
Created new 'BreastCancerClassification' project. ID is: f3ca1ab8-2c87-41f0-928f-c07a5830cea4
Import project zip. ID is: f15fadd9-5f41-

## Extra Cleanup Helpers

In [25]:
# REMOVE ALL CONTEXTS
res = ! cpdctl config context list --output json
contexts = json.loads(res.s)
for context in contexts:
    c_name = context['Name']
    print("Unsetting context: {}".format(c_name))
    ! cpdctl config context unset {c_name}
    
! cpdctl config context list

Unsetting context: cpd_cont_user_0
Unsetting context: cpd_cont_user_1
Unsetting context: cpd_cont_user_2
Unsetting context: cpd_cont_user_3
Unsetting context: cpd_cont_user_4
Unsetting context: cpd_cont_user_5
Unsetting context: cpd_cont_user_6
Unsetting context: cpd_cont_user_7
Unsetting context: cpd_cont_user_8
Unsetting context: cpd_cont_user_9
Unsetting context: cpd_cont_user_10
Unsetting context: cpd_cont_user_11
Unsetting context: cpd_cont_user_12
Unsetting context: cpd_cont_user_13
Unsetting context: cpd_cont_user_14
Unsetting context: cpd_cont_user_15
Unsetting context: cpd_cont_user_16
Unsetting context: cpd_cont_user_17
Unsetting context: cpd_cont_user_18
Unsetting context: cpd_cont_user_19
Unsetting context: cpd_cont_user_20
Unsetting context: cpd_cont_user_21
Unsetting context: cpd_cont_user_22
Unsetting context: cpd_cont_user_23
Unsetting context: cpd_cont_user_24
Unsetting context: cpd_cont_user_25
Unsetting context: cpd_cont_user_26
Unsetting context: cpd_cont_user_27
Un

In [26]:
# REMOVE ALL USERS
res = ! cpdctl config user list --output json
users = json.loads(res.s)
for user in users:
    u_name = user['Name']
    print("Unsetting user: {}".format(u_name))
    ! cpdctl config user unset {u_name}
    
! cpdctl config user list 

Unsetting user: cpd_user_0
Unsetting user: cpd_user_1
Unsetting user: cpd_user_2
Unsetting user: cpd_user_3
Unsetting user: cpd_user_4
Unsetting user: cpd_user_5
Unsetting user: cpd_user_6
Unsetting user: cpd_user_7
Unsetting user: cpd_user_8
Unsetting user: cpd_user_9
Unsetting user: cpd_user_10
Unsetting user: cpd_user_11
Unsetting user: cpd_user_12
Unsetting user: cpd_user_13
Unsetting user: cpd_user_14
Unsetting user: cpd_user_15
Unsetting user: cpd_user_16
Unsetting user: cpd_user_17
Unsetting user: cpd_user_18
Unsetting user: cpd_user_19
Unsetting user: cpd_user_20
Unsetting user: cpd_user_21
Unsetting user: cpd_user_22
Unsetting user: cpd_user_23
Unsetting user: cpd_user_24
Unsetting user: cpd_user_25
Unsetting user: cpd_user_26
Unsetting user: cpd_user_27
Unsetting user: cpd_user_28
Unsetting user: cpd_user_29
Nothing to show.
